<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Reference" data-toc-modified-id="Reference-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Reference</a></span></li></ul></div>

In [1]:
# 1. magic for inline plot
# 2. magic to print version
# 3. magic so that the notebook will reload external python modules
# 4. magic to enable retina (high resolution) plots
# https://gist.github.com/minrk/3301035
%matplotlib inline
%load_ext watermark
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# change default style figure and font size
plt.rcParams['figure.figsize'] = 8, 6
plt.rcParams['font.size'] = 12

# prevent scientific notations
pd.set_option('display.float_format', lambda x: '%.3f' % x)

%watermark -a 'Ethen' -d -t -v -p numpy,pandas,sklearn,matplotlib

Ethen 2020-01-01 11:09:21 

CPython 3.6.4
IPython 7.9.0

numpy 1.16.5
pandas 0.25.0
sklearn 0.21.2
matplotlib 3.1.1


In [5]:
data = pd.read_csv('data_python.csv')
 
visible = data['Visible'].values
visible

array([0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1,
       1, 1, 0, 2, 1, 2, 0, 2, 0, 1, 2, 1, 2, 0, 2, 0, 2, 2, 0, 2, 2, 2,
       0, 0, 1, 0, 1, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 0, 1, 0, 0, 2, 1, 2,
       1, 1, 1, 0, 2, 0, 0, 1, 1, 2, 0, 1, 2, 0, 1, 0, 2, 1, 0, 0, 2, 0,
       1, 0, 2, 1, 2, 1, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1,
       2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 1, 1, 1, 2, 1, 0, 1, 0, 1,
       0, 1, 2, 0, 2, 2, 1, 0, 0, 1, 1, 2, 2, 0, 2, 0, 0, 0, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2,
       0, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2,
       1, 2, 1, 2, 0, 2, 0, 1, 2, 0, 1, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 0, 0, 1, 2, 1, 0, 2, 2, 1, 2, 2, 2, 1, 0, 1, 2, 2, 2, 1, 0,
       1, 0, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 0, 1, 1, 2, 0, 0, 2,
       2, 2, 1, 1, 0, 0, 1, 2, 1, 2, 1, 0, 2, 0, 2, 2, 0, 0, 0, 1, 0, 1,
       1, 1, 2, 2, 0, 1, 2, 2, 2, 0, 1, 1, 2, 2, 0,

2 hidden states, 3 visible state.

In [6]:
# Transition Probabilities
transition_prob = np.array(((0.54, 0.46), (0.49, 0.51)))
 
# Emission Probabilities
# b_jk = probability of emitting symbol k given state j
emission_prob = np.array(((0.16, 0.26, 0.58), (0.25, 0.28, 0.47)))

# Equal Probabilities for the initial distribution
initial_distribution = np.array((0.5, 0.5))

In [15]:
transition_prob

array([[0.54, 0.46],
       [0.49, 0.51]])

In [14]:
emission_prob

array([[0.16, 0.26, 0.58],
       [0.25, 0.28, 0.47]])

In [41]:
(0.08 * 0.54 + 0.08 * 0.49) + (0.125 * 0.54 + 0.125 * 0.49)

0.21115

In [46]:
transition_prob[:, 0] @ result[0]

0.10445

In [50]:
(transition_prob[:, 0] @ result[0]) * emission_prob[0, visible[1]]

0.027157

In [40]:
transition_prob[:, 0]

array([0.54, 0.49])

In [31]:
def forward(visible, transition_prob, emission_prob, initial_distribution):
    n_visible = visible.shape[0]
    n_hidden_state = transition_prob.shape[0]
    result = np.zeros((n_visible, n_hidden_state))
    result[0] = initial_distribution * emission_prob[:, visible[0]]

#     for t in range(1, n_visible):
#         for j in range(n_hidden_state):
#             result[t, j] = emission_prob[j, visible[t]] * (transition_prob[:, j] @ result[t - 1])
    
    for t in range(1, n_visible):
        result[t] = emission_prob[:, visible[t]] * (result[t - 1] @ transition_prob)

    return result


result = forward(visible, transition_prob, emission_prob, initial_distribution)
result

array([[8.00000000e-002, 1.25000000e-001],
       [2.71570000e-002, 2.81540000e-002],
       [1.65069392e-002, 1.26198572e-002],
       [8.75653677e-003, 6.59378003e-003],
       [4.61649960e-003, 3.47369232e-003],
       [2.43311103e-003, 1.83073126e-003],
       [1.28234420e-003, 9.64864889e-004],
       [6.75844805e-004, 5.08520930e-004],
       [3.56196241e-004, 2.68010114e-004],
       [1.87729137e-004, 1.41251652e-004],
       [9.89404851e-005, 7.44450603e-005],
       [5.21454461e-005, 3.92354139e-005],
       [2.74826583e-005, 2.06785741e-005],
       [1.44844194e-005, 1.08984050e-005],
       [7.63384683e-006, 5.74387913e-006],
       [4.02333128e-006, 3.02724551e-006],
       [9.50546790e-007, 9.50495728e-007],
       [5.67842140e-007, 4.33342042e-007],
       [3.01003967e-007, 2.26639558e-007],
       [1.58685405e-007, 1.19402560e-007],
       [8.36344763e-008, 6.29285781e-008],
       [1.97593813e-008, 1.97583215e-008],
       [5.29142730e-009, 5.36649662e-009],
       [1.4

# Reference

- http://www.adeveloperdiary.com/data-science/machine-learning/introduction-to-hidden-markov-model/
- https://github.com/adeveloperdiary/HiddenMarkovModel

- http://www.blackarbs.com/blog/introduction-hidden-markov-models-python-networkx-sklearn/2/9/2017
- https://towardsdatascience.com/markov-chains-and-hmms-ceaf2c854788
- https://sambaiga.github.io/2017/05/03/hmm-intro.html